# Answering Business Questions using SQL

We are going to use the modified version of the [Chinook](https://github.com/lerocha/chinook-database) database which is provided as a SQLite database file to answer some business questions. 

The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store. 


The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables and this is the database diagram:

    

    
   ![title](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

The questions we are going to go through are as follows:

* Which genres sell the most tracks in the USA?
* Which Sales support agents have the highest sales amount?
* Which countries have the most value of sales and the number of customers?
* Which one has more benefit, selling the whole album or the popular tracks?
* Which artist is used in the most playlists?
* How many tracks have been purchased vs not purchased?
* Is the range of tracks in the store reflective of their sales popularity?
* Do protected vs non-protected media types have an effect on popularity?

Let's connect Jupyter Notebook to the database file and get started with our analysis

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Let's look at the list of tables the database contains and also look at some tables.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * 
FROM customer 
Limit 1;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3


## Selecting Albums to Purchase -- Which genres sell the most tracks in the USA?

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

* Regal (Hip-Hop)
* Red Tone (Punk)
* Meteor and the Girls (Pop)
* Slim Jim Bites (Blues)

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll have to find out which genres sell the most tracks in the USA to make a recommendation for the three artists whose albums we should purchase for the store.

In [4]:
%%sql
WITH usa_sales AS 
                 (SELECT *
                 FROM invoice iv
                 INNER JOIN invoice_line il 
                 ON il.invoice_id = iv.invoice_id
                 INNER JOIN customer c 
                 ON c.customer_id = iv.customer_id
                 WHERE c.country = 'USA')


SELECT g.name genre,
       COUNT(t.track_id) tracks_sold,
       CAST(COUNT(t.track_id) AS float)/(SELECT COUNT(*)FROM usa_sales) *100
        AS percentage_sold
FROM usa_sales us
INNER JOIN track t ON t.track_id = us.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY g.genre_id
ORDER BY 2 DESC;


Done.


genre,tracks_sold,percentage_sold
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Another way to solve this:

In [5]:
%%sql

WITH usa_sales AS 
                 (SELECT *, g.name genre
                  FROM invoice iv
                  INNER JOIN invoice_line il 
                  ON il.invoice_id = iv.invoice_id
                  INNER JOIN customer c 
                  ON c.customer_id = iv.customer_id
                  INNER JOIN track t ON t.track_id = il.track_id
                  INNER JOIN genre g ON g.genre_id = t.genre_id
                  WHERE c.country = 'USA')


SELECT genre,
       COUNT(track_id) tracks_sold,
       CAST(COUNT(track_id) AS float)/(SELECT COUNT(*) FROM usa_sales) *100
        AS percentage_sold
FROM usa_sales us
GROUP BY 1
ORDER BY 2 DESC;


Done.


genre,tracks_sold,percentage_sold
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Based on the genres that sell the best in the USA, we can select following three artists whose albums we should purchase for the store:

* Red Tone (Punk: 2nd place)
* Slim Jim Bites (Blues: 5th place)
* Meteor and the Girls (Pop: 8th place)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns (like the hire date) from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [6]:
%%sql

SELECT em.first_name || ' ' || em.last_name employee_name,
       em.hire_date,
       em.title,
       ROUND(SUM(iv.total),2) total_sales
FROM employee em
INNER JOIN customer c
ON em.employee_id = c.support_rep_id
INNER JOIN invoice iv
ON iv.customer_id = c.customer_id
GROUP BY 1
ORDER BY 4 DESC;

Done.


employee_name,hire_date,title,total_sales
Jane Peacock,2017-04-01 00:00:00,Sales Support Agent,1731.51
Margaret Park,2017-05-03 00:00:00,Sales Support Agent,1584.0
Steve Johnson,2017-10-17 00:00:00,Sales Support Agent,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales 
* average order value



In [7]:
%%sql

WITH sales_sum AS 
               (SELECT c.customer_id, 
                       SUM(iv.total) total_val_sales,
                       COUNT(iv.customer_id) num_of_purchase,     
                       c.country
               FROM customer c 
               INNER JOIN invoice iv
               ON c.customer_id = iv.customer_id
           
               GROUP BY 1)



SELECT country, 
       COUNT(customer_id) num_of_customers,
       SUM(total_val_sales) total_sales,
       SUM(CAST(total_val_sales AS float))/COUNT(customer_id) average_val_sales, 
       SUM(CAST(total_val_sales AS float))/SUM(num_of_purchase) average_order    
            
FROM sales_sum
GROUP BY country
ORDER BY total_sales DESC;
             


Done.


country,num_of_customers,total_sales,average_val_sales,average_order
USA,13,1040.49,80.03769230769231,7.942671755725191
Canada,8,535.59,66.94875,7.047236842105264
Brazil,5,427.67999999999995,85.53599999999999,7.011147540983606
France,5,389.07,77.814,7.7814
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.51999999999998,81.83999999999999,8.768571428571429
Portugal,2,185.13,92.565,6.3837931034482756
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Ireland,1,114.83999999999997,114.83999999999997,8.833846153846151


Based on the results, we can conclude that the USA is by far the most important country for sales. Also, based on the average order result data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

The Chinook store is set up in a way that allows a customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.
The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management is currently considering changing its purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

I am going to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

There are two edge cases to consider:

* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done an analysis to confirm that the second case does not happen often, so we can ignore this case also.

In [8]:
%%sql
WITH 
track_album AS 
             (SELECT t.album_id,
                     COUNT(t.track_id) track_num      
              FROM track t
              INNER JOIN album a ON a.album_id = t.album_id
              GROUP BY 1),

orders AS 
            (SELECT i.invoice_id,
                    a.album_id,
                    COUNT(il.track_id) track_num
             FROM invoice i
             INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
             INNER JOIN track t ON t.track_id = il.track_id
             INNER JOIN album a ON a.album_id = t.album_id
             GROUP BY 1,2),

purchase_cat AS 
               (SELECT CASE 
                       WHEN o.track_num = ta.track_num THEN 'album' 
                       ELSE 'single'
                       END AS purchase_cat,
                       o.invoice_id
               FROM orders o
               INNER JOIN track_album ta ON ta.album_id = o.album_id),

edge_free_cat AS 
              (SELECT CASE
                      WHEN COUNT(DISTINCT(purchase_cat))= 2 THEN 'single'
                      ELSE purchase_cat
                      END AS purchase_cat,
                      invoice_id
              FROM purchase_cat
              GROUP BY invoice_id)

SELECT purchase_cat,
       COUNT(invoice_id) amount,
       ROUND(CAST(COUNT(invoice_id) AS FLOAT)/ (SELECT COUNT(invoice_id) FROM edge_free_cat),2) percentage
FROM edge_free_cat
GROUP BY purchase_cat;



Done.


purchase_cat,amount,percentage
album,114,0.19
single,500,0.81


Album purchases account for 19% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

## Which artist is used in the most playlists?

In [9]:
%%sql

SELECT ar.name artist_name, 
       g.name genre,   
       COUNT(DISTINCT p.playlist_id) num_of_playlists
FROM playlist p 
INNER JOIN playlist_track pt ON p.playlist_id = pt.playlist_id
INNER JOIN track t ON t.track_id = pt.track_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
LEFT JOIN genre g ON g.genre_id = t.genre_id

GROUP BY 1
ORDER BY 3 DESC
LIMIT 10;

Done.


artist_name,genre,num_of_playlists
Eugene Ormandy,Classical,7
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,6
Berliner Philharmoniker & Herbert Von Karajan,Classical,6
English Concert & Trevor Pinnock,Classical,6
The King's Singers,Classical,6
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,5
Adrian Leaper & Doreen de Feis,Classical,5
Alberto Turco & Nova Schola Gregoriana,Classical,5
Antal Doráti & London Symphony Orchestra,Classical,5
Barry Wordsworth & BBC Concert Orchestra,Classical,5



Eugene Ormandy is used in 7 playlists. After him, these artists are mentioned in 6 playlists and all of them in the classical genre:

   * Academy of St. Martin in the Fields & Sir Neville Marriner
   * Berliner Philharmoniker & Herbert Von Karajan
   * English Concert & Trevor Pinnock
   * The King's Singers



## How many tracks have been purchased vs not purchased?

In [10]:
 %%sql            

WITH track_invoice AS   (SELECT DISTINCT t.track_id FROM invoice i
                         INNER JOIN invoice_line il 
                         ON il.invoice_id = i.invoice_id
                         INNER JOIN track t 
                         ON t.track_id = il.track_id),

track_only AS           (SELECT *
                         FROM track),

track_purchases AS      (SELECT t.track_id, 
                                t.name, CASE
                            WHEN t.track_id = tiv.track_id THEN 'purchased'
                            ELSE 'not purchased'
                            END AS category
                        FROM track_only t
                        LEFT JOIN track_invoice tiv ON t.track_id = tiv.track_id)

SELECT   category purchase_category, 
         COUNT(*) amount,
         CAST(COUNT (tp.track_id) AS float) /(SELECT COUNT(*) FROM track_purchases) *100
         percentage
FROM track_purchases tp
GROUP BY category
ORDER BY 2 DESC;

Done.


purchase_category,amount,percentage
purchased,1806,51.55580930630887
not purchased,1697,48.44419069369112


Another way to solve this: specify the number of the purchased tracks and total tracks, then the number of not purchased can be calculated

In [11]:
%%sql
SELECT 
    COUNT(DISTINCT track_id) num_purchased_tracks,
     (SELECT COUNT(*) total_num FROM track) - 
        COUNT(DISTINCT track_id) num_not_purchased_tracks
FROM invoice_line;

Done.


num_purchased_tracks,num_not_purchased_tracks
1806,1697



The number of purchased tracks is more than the number of not purchased tracks but about 48% of tracks are not purchased so it seems the range of tracks in the store is not reflective of their sales popularity.



## Do protected vs non-protected media types have an effect on popularity?

In [12]:
%%sql
SELECT *
FROM media_type;

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [13]:
%%sql

SELECT *, CASE
         WHEN name LIKE '%Protected%' THEN 'protected'
         ELSE 'not protected'
         END AS protected_type
FROM media_type

Done.


media_type_id,name,protected_type
1,MPEG audio file,not protected
2,Protected AAC audio file,protected
3,Protected MPEG-4 video file,protected
4,Purchased AAC audio file,not protected
5,AAC audio file,not protected


In [14]:
%%sql

WITH pro_media_type AS (SELECT *, CASE
                                 WHEN name LIKE '%Protected%' THEN 'Protected'
                                 ELSE 'Not Protected'
                                 END AS protected_type
                       FROM media_type),

      inv_sale AS      (SELECT * 
                       FROM invoice iv
                       INNER JOIN invoice_line il 
                       ON il.invoice_id = iv.invoice_id
                       INNER JOIN track t
                       ON t.track_id = il.track_id
                       )


SELECT pmt.protected_type,
       SUM(ins.total) total_sales,
    SUM(CAST(ins.total AS float))/(SELECT SUM(inv_sale.total) FROM inv_sale)*100
    total_sales_percentage

FROM pro_media_type pmt
INNER JOIN inv_sale ins ON pmt.media_type_id = ins.media_type_id
GROUP  BY pmt.protected_type;

Done.


protected_type,total_sales,total_sales_percentage
Not Protected,43362.000000000364,91.28232915824327
Protected,4141.1700000000055,8.717670841756368


Alternative way of solving it:

In [15]:
%%sql

WITH pm_type AS (SELECT * , CASE
                       WHEN mt.name LIKE '%Protected%' THEN 'Protected'
                       ELSE 'Not Protected'
                       END AS protected_type

                FROM invoice iv
                INNER JOIN invoice_line il 
                ON il.invoice_id = iv.invoice_id
                INNER JOIN track t
                ON t.track_id = il.track_id
                INNER JOIN media_type mt on mt.media_type_id = t.media_type_id)


SELECT protected_type,
       SUM(total) total_sales,
    SUM(CAST(total AS float))/(SELECT SUM(total) FROM pm_type)*100
    total_sales_percentage
    
FROM pm_type
GROUP  BY protected_type;

Done.


protected_type,total_sales,total_sales_percentage
Not Protected,43362.000000000364,91.28232915824327
Protected,4141.1700000000055,8.717670841756368


The above table shows that the not-protected media type has allocated more than 90% of the sales to itself. So not-protected media is more popular than the protected ones.

# Conclusion
Analyzing the Chinook database, we have figured out that Rock, Punk, and Metal genre sell the most tracks in the USA respectively.

The USA has the most number of customers and the total value of sales. Canada and Brazil are at the second and third positions respectively. The most average sale per customer and the most average order are in the Czech Republic.

Eugene Ormandy in Classical genre is used in the highest number of playlists. It is also interesting to know that 48% of tracks are not purchased so it seems the range of tracks in the store is not reflective of their sales popularity.

It is also understood that the not-protected tracks have a significantly higher number of sales.

So these recommendations come out of the analysis:

* to sell the music tracks in Rock, Punk, and Metal more than the others.
* to make playlists with the artists in the above genre
* to sell the tracks with the not-protected media type
* to sell popular tracks rather than the whole album